# Homework 1 Part 4

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri
#### Notebooks Prepared By: Zeinab Sadat Taghavi, Hamed Jamshidian, Seyed Mohammad Reza Modarres

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between `## Your code begins ##` and `## Your code ends ##`) with the appropriate details.


#### Reihaneh Halvaei

## Introduction

In this notebook you have to use [Adapter Hub](https://docs.adapterhub.ml/overview.html) to define and train your adapters.

### Requirements

In [1]:
%%capture
! pip install datasets adapter-transformers

### Imports

In [2]:
from tqdm.notebook import tqdm
from IPython import display

import numpy as np
import pandas as pd
import math

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import T5TokenizerFast, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

### Constants

We will use `t5-small` as our base model from Hugging Face ([HF_Link](https://huggingface.co/t5-small)). And we use `32` as our adapter bottleneck size.

In [3]:
#####################################
###### DO NOT CHANGE THIS CELL ######
#####################################

BASE_MODEL_NAME = 't5-small'

BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Dataset

### Load dataset

`imdb` dataset is a famouns NLP sentiment dataset. Each row of data is either `negative` or `positive`.

In [4]:
dataset = load_dataset('imdb')
dataset.pop('unsupervised')
print(dataset)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


### Define related functions

Because `T5` model is a sequence to sequence model we should map our labels to label_names before training and doing vice versa duing calculating metrics.

The functions `id2label` and `label2id` are defined to do this.

In [5]:
def id2label(ids):
    label_names = ['negative', 'positive']
    return [label_names[id] for id in ids]

def label2id(labels):
    label_names_dict = {
        'negative': 0,
        'positive': 1
    }
    return [
        label_names_dict.get(label, 2)
        for label in labels
    ]

# Tokenizer

### Load tokenizer

In [6]:
tokenizer = T5TokenizerFast.from_pretrained(BASE_MODEL_NAME)

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

(…)ce.co/t5-small/resolve/main/spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…).co/t5-small/resolve/main/tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

### Process dataset using tokenizer

In this step we will getting our dataset ready for training.

We preprocess tokenize our `text` and `label`.

In [7]:
def preprocess_input(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    return text

def map_function(row):
    processed_input = [
        preprocess_input(text)
        for text in row['text']
    ]
    input_info = tokenizer(processed_input, truncation=True, max_length=256)
    output_info = tokenizer(id2label(row['label']))
    return {
        **input_info,
        'labels': output_info.input_ids
    }


dataset = dataset.map(map_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# Model

### Load model and create adapter

In next part create `PfeifferConfig` by considering `BOTTLENECK_SIZE`.

Complete the next cell using methods of `train_adapter` and `add_adapter`.

Report final test data performance.s

Read [this page](https://docs.adapterhub.ml/training.html) for more information.

In [8]:
from transformers import PfeifferConfig

BOTTLENECK_SIZE = 8

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

######### Your code begins #########
####################################
####################################

adapter_config = PfeifferConfig(reduction_factor=BOTTLENECK_SIZE)

model.add_adapter(config=adapter_config, adapter_name="imdb_adapterH_1")

(…)ace.co/t5-small/resolve/main/config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

(…)mall/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Train and evaluate

Complete next part to train your peft using `AdapterTrainer`.

In [9]:
from transformers import TrainingArguments, AdapterTrainer

col_fn = DataCollatorForSeq2Seq(
    tokenizer, return_tensors='pt', padding='longest',
)

training_args = TrainingArguments(
    './temp',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE
)

######### Your code begins #########
####################################
####################################
from transformers import EvalPrediction


def compute_accuracy(p: EvalPrediction):
    labels_ids = p.label_ids
    pred_ids = p.predictions[0]

    #preds = np.argmax(p.predictions, axis=1)
    return {"acc": (labels_ids == pred_ids).mean()}


def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels


model.train_adapter("imdb_adapterH_1")

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=col_fn,
    compute_metrics=compute_accuracy,#lambda p: {'acc': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)


In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7820
  Number of trainable parameters = 793344
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to enc

Epoch,Training Loss,Validation Loss,Acc
1,0.248600,0.127948,0.946280
2,0.139200,0.125319,0.948660
3,0.129400,0.118241,0.951500
4,0.123700,0.117295,0.953100
5,0.118200,0.120002,0.951660
6,0.113100,0.115536,0.954300
7,0.108900,0.118900,0.954020
8,0.104900,0.122664,0.953740
9,0.102100,0.118052,0.955320
10,0.102300,0.115300,0.955700


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 32
Saving model checkpoint to ./temp/checkpoint-782
Configuration saved in ./temp/checkpoint-782/imdb_adapterH_1/adapter_config.json
Module weights saved in ./temp/checkpoint-782/imdb_adapterH_1/pytorch_adapter.bin
Configuration saved in ./temp/checkpoint-782/imdb_adapterH_1/head_config.json
Module weights saved in ./temp/checkpoint-782/imdb_adapterH_1/pytorch_model_head.bin
tokenizer config file saved in ./temp/checkpoint-782/tokenizer_config.json
Special tokens file saved in ./temp/checkpoint-782/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If

TrainOutput(global_step=7820, training_loss=0.1290551305122083, metrics={'train_runtime': 3615.1665, 'train_samples_per_second': 69.153, 'train_steps_per_second': 2.163, 'total_flos': 1.722236928e+16, 'train_loss': 0.1290551305122083, 'epoch': 10.0})

In [11]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 32


{'eval_loss': 0.11529955267906189,
 'eval_acc': 0.9557,
 'eval_runtime': 121.0094,
 'eval_samples_per_second': 206.595,
 'eval_steps_per_second': 6.462,
 'epoch': 10.0}

# Bottleneck effect

Check the model performance with `BOTTLENECK_SIZE = 1` and report the number.

In [12]:
from transformers import PfeifferConfig

BOTTLENECK_SIZE = 1

model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

######### Your code begins #########
####################################
####################################

adapter_config = PfeifferConfig(reduction_factor=BOTTLENECK_SIZE)

model.add_adapter(config=adapter_config, adapter_name="imdb_adapterH_2")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefi

In [13]:

training_args = TrainingArguments(
    './temp2',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE
)

######### Your code begins #########
####################################
####################################

model.train_adapter("imdb_adapterH_2")


trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=col_fn,
    compute_metrics=compute_accuracy,#lambda p: {'acc': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7820
  Number of trainable parameters = 6303744


Epoch,Training Loss,Validation Loss,Acc
1,0.181700,0.126550,0.946880
2,0.129900,0.139725,0.946280
3,0.114800,0.114096,0.954000
4,0.101600,0.121345,0.952720
5,0.089800,0.117991,0.955340
6,0.076700,0.121513,0.955260
7,0.066100,0.133901,0.953940
8,0.058400,0.154362,0.953700
9,0.050800,0.147865,0.954500
10,0.047400,0.153206,0.955200


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 32
Saving model checkpoint to ./temp2/checkpoint-782
Configuration saved in ./temp2/checkpoint-782/imdb_adapterH_2/adapter_config.json
Module weights saved in ./temp2/checkpoint-782/imdb_adapterH_2/pytorch_adapter.bin
Configuration saved in ./temp2/checkpoint-782/imdb_adapterH_2/head_config.json
Module weights saved in ./temp2/checkpoint-782/imdb_adapterH_2/pytorch_model_head.bin
tokenizer config file saved in ./temp2/checkpoint-782/tokenizer_config.json
Special tokens file saved in ./temp2/checkpoint-782/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: t

TrainOutput(global_step=7820, training_loss=0.09172855972329064, metrics={'train_runtime': 3987.5168, 'train_samples_per_second': 62.696, 'train_steps_per_second': 1.961, 'total_flos': 1.933836288e+16, 'train_loss': 0.09172855972329064, 'epoch': 10.0})

In [15]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 32


{'eval_loss': 0.1532062292098999,
 'eval_acc': 0.9552,
 'eval_runtime': 127.5614,
 'eval_samples_per_second': 195.984,
 'eval_steps_per_second': 6.13,
 'epoch': 10.0}